[Matt Corallo](https://github.com/TheBlueMatt) has [asked about the Privacy implications of implicitly signaling drain](https://lists.linuxfoundation.org/pipermail/lightning-dev/2022-September/003690.html) and demand on channels when doing flow controll valves by setting asymmetric `htlc_maximum_size` pairs on a channel which we suggested to do in [this notebook](https://github.com/lnresearch/Flow-Control-on-Lightning-Network-Channels-with-Drain-via-Control-Valves/blob/main/htlc_maximum_msat%20as%20a%20valve%20for%20flow%20control%20on%20the%20Lightnig%20network.ipynb) and that [Mailinglist post](https://lists.linuxfoundation.org/pipermail/lightning-dev/2022-September/003686.html) where we shared the [BitMEX blog post](https://blog.bitmex.com/the-power-of-htlc_maximum_msat-as-a-control-valve-for-better-flow-control-improved-reliability-and-lower-expected-payment-failure-rates-on-the-lightning-network/)

This notebook basically is an adopted version of [experimental idea one in the oritinal notebook](https://github.com/lnresearch/Flow-Control-on-Lightning-Network-Channels-with-Drain-via-Control-Valves/blob/main/htlc_maximum_msat%20as%20a%20valve%20for%20flow%20control%20on%20the%20Lightnig%20network.ipynb). It shall indicate in a small simulation / random walk of payments on channels with capacity of roughly 1M sats how much information about the historic distribution of liquidity will leak if a certain drain is induced from a current `htlc_maximum_size` pair. This is done by measureing the entropy of the resulting distribution and comparing it against the uniform distribution. 

## Acknowledgements 
This notebook was creted by [Rene Pickhardt](https://ln.rene-pickhardt.de) as a reply to Matt Corallo's question about privacy implications. 

The research of Rene Pickhardt is funded through [NTNU](https://www.ntnu.no/) & [BitMEX](https://blog.bitmex.com/rene-pickhardt/) and various doners at https://donation.rene-pickhardt.de and https://www.patreon.com/renepickhardt 

In [1]:
from math import log2 as log
import random

c = 2**20 - 1 # roughly 1 Million sats as capacity
max_a = 500_000 #htlc_maximum_msats in direction 1
max_b = 500_000 #htlc_maximum_msats in direction 2
A = 1000_000 #Alices liquidity
N = 10_000_000 #number of payments in the random walk after which we measure the entropy

for x in range(1,10):
    drain = 0.5 + x * 0.05


    ## do a small random walk of N payments on the channel
    liquidity_values = {i:0 for i in range(c+1)}
    for i in range(N):
        if random.random() > drain:
            size = random.randint(1,max_b)
            if A - size >= 0:
                A -= size
        else:
            size = random.randint(1,max_a)
            if A + size <= c:
                A += size
        liquidity_values[A]+=1

    # probability of liquidity values
    pdf = [liquidity_values[i]/N for i in range(c+1)]
    
    # entropy of the observed probability distribution
    H = -sum(log(p)*p for p in pdf if p > 0)
    
    print("Assume we learnt a drain of {:4.2f}".format(drain))
    print("=================================")
    print("Entropy of uniform: {:4.2f} bits.\nEntropy of the channel with drain: {:4.2f} bits\nInformation gain: {:4.2f} bits\n".format(log(c+1), H, log(c+1) - H))


Assume we learnt a drain of 0.55
Entropy of uniform: 20.00 bits.
Entropy of the channel with drain: 19.84 bits
Information gain: 0.16 bits

Assume we learnt a drain of 0.60
Entropy of uniform: 20.00 bits.
Entropy of the channel with drain: 19.75 bits
Information gain: 0.25 bits

Assume we learnt a drain of 0.65
Entropy of uniform: 20.00 bits.
Entropy of the channel with drain: 19.62 bits
Information gain: 0.38 bits

Assume we learnt a drain of 0.70
Entropy of uniform: 20.00 bits.
Entropy of the channel with drain: 19.42 bits
Information gain: 0.58 bits

Assume we learnt a drain of 0.75
Entropy of uniform: 20.00 bits.
Entropy of the channel with drain: 19.17 bits
Information gain: 0.83 bits

Assume we learnt a drain of 0.80
Entropy of uniform: 20.00 bits.
Entropy of the channel with drain: 18.86 bits
Information gain: 1.14 bits

Assume we learnt a drain of 0.85
Entropy of uniform: 20.00 bits.
Entropy of the channel with drain: 18.46 bits
Information gain: 1.54 bits

Assume we learnt a d